In [1]:
# Combine multiple dicts into one

import numpy as np
from copy import deepcopy
import numbers

def compare(a, b):
    """ Compare if two variables have equal values, it can compare only 
        numbers, lists, and numpy arrays. Everythings else will return False. """
    if isinstance(a, numbers.Number) and isinstance(b, numbers.Number):
        return a == b
    elif isinstance(a, dict) and isinstance(b, dict):
        return a == b
    elif isinstance(a, list) and isinstance(b, list):
        return (np.array(a) == np.array(b)).all()
    elif isinstance(a, np.ndarray) and isinstance(b, np.ndarray):
        return np.array_equal(a, b)
    else:
        return False
    
def merge_dicts(a, b, path=None, bail_on_conflict=True):
    """ Merges two nested dictionaries. 
        b is merged into a, altering a and returning it.
        Usage: d = merge_dicts(a, b) 
        Reports error if there is conflict.
        a, b : the two dictionaries to merge
        path : Used internally to keep track of the nested dict keys path for reporting errors
        bail_on_conflict : What to do if a conflict is found, 
                           True throws Exception, 
                           False keeps the value of dictionary a """
    if path is None: path = [] # Keep track of the nested dict keys path for reporting errors
    for key in b:
        if key in a:
            if isinstance(a[key], dict) and isinstance(b[key], dict):
                merge_dicts(a[key], b[key], path + [str(key)], bail_on_conflict)
            #elif a[key] == b[key]:
            elif compare(a[key], b[key]):
                pass # same leaf value
            else:
                if bail_on_conflict:
                    raise Exception('Conflict at %s' % '.'.join(path + [str(key)]))
                else:
                    print('Conflict at {}, {}.'.format('.'.join(path + [str(key)]), 'kept value from dictionary \'a\''))
        else:
            a[key] = b[key]
    return a

# Example
# a = {'a': {'1': 1, '2': 2}, 'b': {'11': 1, '12': 2}}
# b = {'c': {'21': 1, '22': 2}, 'd': {'31': 1, '32': 2}}
# c = {'c': {'21': 41, '42': 42}}
# d = merge_dicts(deepcopy(a), b)
# e = merge_dicts(d, c, bail_on_conflict=False)


In [2]:
# Load the data (search_dispersion_list dict) from previous run and new additional runs
filename_path      = 'data/3_parameters_results_correct_Nl_range_combined_01Ded2022/' # The path
input_filename_path      = filename_path + 'individual/' # The path
# The data collected on 19 November 2022
filename_results_npz_1 = input_filename_path + 'path_analysis_calculation_results_3parameters.npz'
# The additional data collected on 1 December 2022
filenames_results_npz = ['path_analysis_calculation_results_3parameters_new_batch_0.0.npz', 
                         'path_analysis_calculation_results_3parameters_new_batch_0.001.npz', 
                         'path_analysis_calculation_results_3parameters_new_batch_0.002.npz', 
                         'path_analysis_calculation_results_3parameters_new_batch_0.003.npz', 
                         'path_analysis_calculation_results_3parameters_new_batch_0.004.npz', 
                         'path_analysis_calculation_results_3parameters_new_batch_0.005.npz', 
                         'path_analysis_calculation_results_3parameters_new_batch_0.006.npz', 
                         'path_analysis_calculation_results_3parameters_new_batch_0.007.npz', 
                         'path_analysis_calculation_results_3parameters_new_batch_0.008.npz', 
                         'path_analysis_calculation_results_3parameters_new_batch_0.009.npz', 
                         'path_analysis_calculation_results_3parameters_new_batch_0.01.npz', 
                         'path_analysis_calculation_results_3parameters_new_batch_0.015.npz', 
                         'path_analysis_calculation_results_3parameters_new_batch_0.02.npz', 
                         # Additional nosie values
                         'path_analysis_calculation_results_3parameters_new_batch_noise0.0055_0.0065.npz', 
                         # Partial extra parameter values not good match to ant data
                         #'path_analysis_calculation_results_3parameters_new_batch-Nl0.023r-0.019.npz' # Nl=0.023 r=-0.019
                        ]

# Save the combined data in
output_results_filename_npz = filename_path + 'path_analysis_calculation_results_3parameters.npz'

# Load the data of the previous run
search_dispersion_dict_1 = np.load(filename_results_npz_1, allow_pickle=True)['arr_0'][()]

# Make a copy
merged_dict = deepcopy(search_dispersion_dict_1)

# Add to the dictionary the entries from the new runs
for filename in filenames_results_npz:
    search_dispersion_dict_2 = np.load(input_filename_path + filename, allow_pickle=True)['arr_0'][()]
    merged_dict = merge_dicts(merged_dict, search_dispersion_dict_2, bail_on_conflict=True)

# Store data to file
# Store the results_dict dict which contains the calculated path statistics
# The data in this file will have the structure dict[wait_noise_sd_str][mem_Nl_str][mem_r_str][measure] = [list of values]
np.savez(output_results_filename_npz, merged_dict)


In [7]:




# Load the data (search_dispersion_list dict) from previous run and new additional runs
filename_path      = 'data/3_parameters_results_extra_search_space_05Ded2022/' # The path
input_filename_path      = filename_path + 'individual/' # The path
# The data collected on 19 November 2022
filename_results_npz_1 = input_filename_path + 'path_analysis_calculation_results_3parameters.npz'
# The additional data collected on 1 December 2022
filenames_results_npz = ['path_analysis_calculation_results_3parameters_new_batch_extra2.npz', 
                         'path_analysis_calculation_results_3parameters_new_batch_extra3.npz', 
                         'path_analysis_calculation_results_3parameters_new_batch_noise0.0055_0.0065_0.0075.npz'
                        ]

# Save the combined data in
output_results_filename_npz = filename_path + 'path_analysis_calculation_results_3parameters.npz'

# Load the data of the previous run
search_dispersion_dict_1 = np.load(filename_results_npz_1, allow_pickle=True)['arr_0'][()]

# Make a copy
merged_dict = deepcopy(search_dispersion_dict_1)

# Add to the dictionary the entries from the new runs
for filename in filenames_results_npz:
    search_dispersion_dict_2 = np.load(input_filename_path + filename, allow_pickle=True)['arr_0'][()]
    merged_dict = merge_dicts(merged_dict, search_dispersion_dict_2, bail_on_conflict=True)

# Store data to file
# Store the results_dict dict which contains the calculated path statistics
# The data in this file will have the structure dict[wait_noise_sd_str][mem_Nl_str][mem_r_str][measure] = [list of values]
np.savez(output_results_filename_npz, merged_dict)






In [6]:
# Load the data (search_dispersion_list dict) from individual files and combine them in a file
filename_path      = 'data/3_parameters_results_correct_Nl_range_combined_01Ded2022/' # The path
input_filename_path      = filename_path + 'individual/' # The subdir

# The data collected on 1 December 2022
input_filenames_results_npz = ['path_analysis_calculation_results_3parameters_distance_scaling_factor2.87_noise0.0--0.004.npz', 
                               'path_analysis_calculation_results_3parameters_distance_scaling_factor2.87_noise0.005--0.007.npz', 
                               'path_analysis_calculation_results_3parameters_distance_scaling_factor2.87_noise0.008--0.02.npz']

# Save the dictionary with the combined data in
output_results_filename_npz = filename_path + 'path_analysis_calculation_results_3parameters.npz'

# Resulting dictionary
merged_dict = {}

# Add to the dictionary the entries from the individual files
for filename in input_filenames_results_npz:
    dict_i = deepcopy(np.load(input_filename_path + filename, allow_pickle=True)['arr_0'][()])
    merged_dict = merge_dicts(merged_dict, dict_i, bail_on_conflict=True)

# Store data to file
# Store the results_dict dict which contains the calculated path statistics
# The data in this file will have the structure dict[wait_noise_sd_str][mem_Nl_str][mem_r_str][measure] = [list of values]
np.savez(output_results_filename_npz, merged_dict)


In [7]:
merged_dict['0.0'].keys()

dict_keys(['0.0', '0.001', '0.01', '0.016', '0.018', '0.019', '0.02', '0.021', '0.022', '0.023', '0.024', '0.025', '0.03', '0.04', '0.05', '0.06', '0.08', '0.1', '0.12', '0.14', '0.16', '0.18', '0.2', '0.3', '0.5', '0.999', '1.3'])

In [8]:
merged_dict['0.002']['0.005'].keys()

KeyError: '0.005'

In [10]:
# Load the stored file
#filename_path      = 'data/3_parameters_results/'
#output_results_filename_npz = filename_path + 'path_analysis_calculation_results_3parameters.npz'
#merged_dict = np.load(output_results_filename_npz, allow_pickle=True)['arr_0'][()]


In [ ]:
# Check dict for existence of values

#for wait in ['0.0', '1.0', '24.0', '48.0', '96.0', '144.0', '192.0', '240.0', '288.0', '336.0', '384.0', '432.0']: # 12
#    if not wait in merged_dict:
#        print('WARNING: key {} not in dictionary merged_dict'.format(wait))
for noise in ['0.0', '0.001', '0.002', '0.003', '0.004', '0.005', '0.0055', '0.006', '0.0065', '0.007', '0.008', '0.009', '0.01', '0.015', '0.02']: # 13
    if not noise in merged_dict:
        print('WARNING: key noise={} not in dictionary merged_dict'.format(noise))
    else:
        for mem_Nl in ['0.0', '0.001', '0.01', '0.016', '0.018', '0.019', '0.02', '0.021', '0.022', '0.025', '0.03', '0.04', '0.05', '0.06', '0.08', '0.1', '0.12', '0.14', '0.16', '0.18', '0.2', '0.3', '0.5', '0.999', '1.3']: # 25
        #for mem_Nl in ['0.0', '0.0005', '0.00075', '0.001', '0.00125', '0.0015', '0.002', '0.0025', '0.003', '0.0035', '0.004', '0.0045', '0.005', '0.0055', '0.006', '0.0065', '0.007', '0.0075', '0.008', '0.0085', '0.009', '0.0095', '0.01']:
            if not mem_Nl in merged_dict[noise]:
                print('WARNING: key mem_Nl={} not in dictionary merged_dict[{}]'.format(mem_Nl, noise))
            else:
                for mem_r in ['-0.008', '-0.01', '-0.012', '-0.014', '-0.015', '-0.016', '-0.017', '-0.018', '-0.019', '-0.02', '-0.021', '-0.022', '-0.023', '-0.024', '-0.025', '-0.026', '-0.027', '-0.028', '-0.029', '-0.03', '-0.031', '-0.032', '-0.035', '-0.04', '-0.045', '-0.05', '-0.1']: # 27
                #for mem_r in ['-0.033', '-0.034', '-0.035', '-0.036', '-0.037', '-0.038', '-0.039', '-0.04', '-0.041', '-0.042']:
                    if not mem_r in merged_dict[noise][mem_Nl]:
                        print('WARNING: mem_r=key {} not in dictionary merged_dict[{}][{}]'.format(mem_r, noise, mem_Nl))


In [4]:
# Fill in the missing data for an r with Nl=0
# We consider merged_dict['0.0']['0.0']['-0.008'] is holding the archetypical value for the Nl=0 condition
mem_Nl = '0.0'
for noise in ['0.0', '0.001', '0.002', '0.003', '0.004', '0.005', '0.0055', '0.006', '0.0065', '0.007', '0.008', '0.009', '0.01', '0.015', '0.02']: # 13
    for mem_r in ['-0.008', '-0.01', '-0.012', '-0.014', '-0.015', '-0.016', '-0.017', '-0.018', '-0.019', '-0.02', '-0.021', '-0.022', '-0.023', '-0.024', '-0.025', '-0.026', '-0.027', '-0.028', '-0.029', '-0.03', '-0.031', '-0.032', '-0.035', '-0.04', '-0.045', '-0.05', '-0.1']: # 27
        #if not mem_r in merged_dict[noise][mem_Nl]:
        #    print('Was missing a value for dictionary merged_dict[{}][{}][{}]'.format(noise, mem_Nl, mem_r))
        #    print('Set to the value of merged_dict[{}][0.0][-0.008]'.format(noise))
        #    merged_dict[noise][mem_Nl][mem_r] = deepcopy(merged_dict[noise][mem_Nl]['-0.008'])
        if mem_r not in merged_dict[noise][mem_Nl]:
            merged_dict[noise][mem_Nl][mem_r] = deepcopy(merged_dict[noise][mem_Nl]['-0.008'])


In [6]:
# Store data to file
# Store the results_dict dict which contains the calculated path statistics
# The data in this file will have the structure dict[wait_noise_sd_str][mem_Nl_str][mem_r_str][measure] = [list of values]
np.savez(output_results_filename_npz, merged_dict)


In [26]:
import numpy as np
filename_path      = 'data/3_parameters_results_correct_Nl_range_combined_01Ded2022/' # The path

f1 = filename_path +  'with_factor_distance_scaling_factor_3/' + 'path_analysis_calculation_results_3parameters.npz'
f2 = filename_path + 'with_factor_distance_scaling_factor_2.87/' + 'path_analysis_calculation_results_3parameters_distance_scaling_factor2.87_noise0.0--0.004.npz'

d1 = np.load(f1, allow_pickle=True)['arr_0'][()]
d2 = np.load(f2, allow_pickle=True)['arr_0'][()]

In [27]:
d1['0.0']['0.0']['-0.008'].keys()

dict_keys(['Wait', 'Dispersion', 'Distance_Median', 'Distance_Median_tocntr', 'Distance_Median_toturn', 'Distance_Dispersion', 'Distance_Dispersion_tocntr', 'Distance_Dispersion_toturn', 'Exit_Angle_Median', 'Exit_Angle_Median_Dev', 'Angle_Dispersion', 'Angle_Dispersion_tocntr', 'Angle_Dispersion_toturn', 'Distance_hypot', 'Distance_hypot_tocntr', 'Distance_hypot_toturn', 'Mean_vector_v', 'Mean_vector_v_proj', 'Distance_to_Nest_Dispersion', 'Distance_to_Nest_Dispersion_tocntr', 'Distance_to_Nest_Dispersion_toturn'])

In [37]:
measure = 'Distance_Dispersion'

In [49]:
from deepdiff import DeepDiff
for measure in d1['0.0']['0.0']['-0.008'].keys():
    print()
    print(measure + ':')
    ddiff = DeepDiff(d1['0.0']['0.0']['-0.008'][measure], d2['0.0']['0.0']['-0.008'][measure], significant_digits=2)
    #print(ddiff)
    if ddiff != {}:
        print('  They differ')


Wait:

Dispersion:
  They differ

Distance_Median:
  They differ

Distance_Median_tocntr:
  They differ

Distance_Median_toturn:
  They differ

Distance_Dispersion:
  They differ

Distance_Dispersion_tocntr:
  They differ

Distance_Dispersion_toturn:
  They differ

Exit_Angle_Median:
  They differ

Exit_Angle_Median_Dev:
  They differ

Angle_Dispersion:

Angle_Dispersion_tocntr:

Angle_Dispersion_toturn:
  They differ

Distance_hypot:
  They differ

Distance_hypot_tocntr:
  They differ

Distance_hypot_toturn:
  They differ

Mean_vector_v:
  They differ

Mean_vector_v_proj:
  They differ

Distance_to_Nest_Dispersion:
  They differ

Distance_to_Nest_Dispersion_tocntr:
  They differ

Distance_to_Nest_Dispersion_toturn:
  They differ


In [34]:
d1['0.0']['0.0']['-0.008'].keys()
d1['0.0']['0.0']['-0.008']['Distance_Median']

[13.225390515729885,
 12.959980994941095,
 12.935616917340298,
 13.50332058085987,
 13.275778591854575,
 13.289338595373493,
 13.421196654906103,
 13.447920596226867,
 13.210120416180978,
 13.416809497642124,
 13.509258619779438,
 13.374571434541199]

In [35]:
d2['0.0']['0.0']['-0.008'].keys()
d2['0.0']['0.0']['-0.008']['Distance_Median']

[12.652290260048257,
 12.398381818493647,
 12.375073517588884,
 12.918176689022609,
 12.70049485287421,
 12.71346725624064,
 12.83961146652684,
 12.86517737039037,
 12.637681864813135,
 12.835414419410965,
 12.923857412922327,
 12.795006672377749]

In [46]:
from numpy.random import normal
r = normal(loc=10, scale=1, size=100)

print(np.mean(r), np.mean(r*0.9), np.std(r), np.std(r*0.9))

10.040645864746939 9.036581278272244 0.9997496400827489 0.899774676074474


In [47]:
0.9997496400827489*0.9

0.899774676074474

In [48]:
{} == {}

True